In [1]:
import sys
import os 
import numpy as np
import pandas as pd
from scipy.stats.qmc import Sobol
from SALib.util import scale_samples
from SALib.sample import sobol
from SALib.analyze import sobol as sobol_analyze
import random

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.sampling.sampler import get_polarisation_curve_samples, build_fixed_parameters, sample_parameters, PARAMETER_RANGES
from configuration.settings import current_density_parameters, physical_parameters, computing_parameters, operating_inputs
from modules.display_modules import plot_lambda
from model.AlphaPEM import AlphaPEM

###  **We define different sets of bounds depending on the experiments we wan to run**

In [ ]:
##== This configuration uses the bounds originaly defined with Luis
PARAMETER_RANGES

{'Tfc': (333, 363),
 'Pa_des': (130000.0, 300000.0),
 'Pc_des': None,
 'Sc': (1.1, 3),
 'Phi_c_des': (0.1, 0.7),
 'epsilon_gdl': (0.55, 0.8),
 'tau': (1.0, 4.0),
 'epsilon_mc': (0.15, 0.4),
 'epsilon_c': (0.15, 0.3),
 'e': [3, 4, 5],
 'Re': (5e-07, 5e-06),
 'i0_c_ref': (0.001, 500),
 'kappa_co': (0.01, 40),
 'kappa_c': (0, 100),
 'a_slim': (0, 1),
 'b_slim': (0, 1),
 'a_switch': (0, 1)}

In [ ]:
##== This configurations were from the first hyperbox optimization 13.06.2025
PARAMETER_RANGES_hyper_box ={'Tfc':( 340.974 , 351.485266662598 ),'Pa_des':( 2e+05 , 256999.999999997 ),
                             'Sa':( 1.1 , 3 ),'Phi_a_des':( 0.31 , 1 ),'epsilon_gdl':( 0.60624999775 , 0.701 ),
                             'tau':( 1.02 , 2.689 ),'epsilon_mc':( 0.2325 , 0.4 ),'epsilon_c':( 0.245 , 0.286 ),
                             'e':( 3 , 5 ),'Re':( 5.7e-07 , 4.2125e-06 ),'i0_c_ref':( 1.3955 , 2.79 ),
                             'kappa_co':( 15.0585 , 37.821626953125 ),'kappa_c':( 0 , 1.88512650296844 ),
                             'a_slim':( 0.056 , 0.900493421521382 ),'b_slim':( 0.0550000005 , 0.542000003 ),
                             'a_switch':( 0.499999995 , 0.842 )}

In [ ]:
##==Those bounds flexibilize the hyperbox allowing up to 80% of probability for a suitable configuration - 15.06.2025
hyperbox_80 = pd.read_csv("../data/raw//hyperbox_prim_150625.csv", sep=';')
hyperbox_dict_80 = dict(zip(hyperbox_80['parameter'], zip(hyperbox_80['lower'], hyperbox_80['upper'])))

In [ ]:
##== boundaries keeping operational conditions unrestricted. The undertermined conditions are restrictied by the hyperbox (hyperbox_prim_150625)
PARAMETER_RANGES_hyperbox_PRIM_2 = PARAMETER_RANGES.copy()
keys_to_filter = ['epsilon_gdl','tau', 'epsilon_mc', 'epsilon_c', 'e', 'Re', 'i0_c_ref', 'kappa_co', 'kappa_c', 'a_slim', 'b_slim', 'a_switch']
restricted_undertermined_conditions = {k: hyperbox_dict_80[k] for k in keys_to_filter if k in hyperbox_dict_80}
PARAMETER_RANGES_hyperbox_PRIM_2.update(restricted_undertermined_conditions)

In [ ]:
##==Those bounds flexibilize the hyperbox allowing up to 80% of probability for a suitable configuration (second configuration) - 15.06.2025
hyperbox_80_second = pd.read_csv("../data/raw//hyperbox_prim_150625_second_configuration.csv", sep=';')
hyperbox_80_second = dict(zip(hyperbox_80_second['parameter'], zip(hyperbox_80_second['lower'], hyperbox_80_second['upper'])))
keys_to_delete = ['Sa', 'Phi_a_des']
for key in keys_to_delete:
    hyperbox_80_second.pop(key, None)

In [4]:
def make_bounds_exclusive(bounds, epsilon=1e-8):
    return [(low + epsilon, high - epsilon) for (low, high) in bounds]

def prepare_problem(PARAMETER_, dependent_param="Pc_des"):
    independent_params = {
        k: v for k, v in PARAMETER_.items()
        if k != dependent_param
    }
    # Add dummy variable to later compute dependent_param
    z_key = f"{dependent_param}"
    independent_params[z_key] = (0, 1)

    bounds = [v if isinstance(v, tuple) else [min(v), max(v)] for v in independent_params.values()]
    bounds = make_bounds_exclusive(bounds)

    problem = {
        "num_vars": len(independent_params),
        "names": list(independent_params.keys()),
        "bounds": bounds,
        "groups": list(independent_params.keys())  # optional but useful for group-based SA
    }
    return problem

problem = prepare_problem(PARAMETER_=hyperbox_80_second,dependent_param = "Pc_des")


`problem` is a requiered object to run the sobol sequences sampling.We use the library SALib for the [Sobol sampling](https://salib.readthedocs.io/en/latest/api/SALib.sample.html#module-SALib.sample.sobol). From the documentation we have "The Sobol’ sequence is a popular quasi-random low-discrepancy sequence used to generate uniform samples of parameter space."

In [ ]:
seed = 33
N=5
np.random.seed(seed)
random.seed(seed)
samples = sobol.sample(problem, N=N, calc_second_order= False, scramble= True, seed=seed)
pd.DataFrame(samples).to_pickle("../data/raw/sobol_sampling_configurations_hyper_box_undertermined_restricted.pkl")

In [12]:
samples.shape

(95, 17)

We define the values for `Pc_des`,`Sa` and `Phi_a_des`. According to our conversations with Luis they should have fixed values.

In [27]:
samples = pd.read_pickle('../data/raw/sobol_sampling_configurations_hyper_box_undertermined_restricted.pkl')
samples.columns = problem['names']
samples['Pc_des'] = samples['Pa_des']-20000
samples['e'] = np.round(samples['e'])
samples['Sa'] = 1.3
samples['Phi_a_des'] = 0.5

In [28]:
samples.head()

,Tfc,Pa_des,Sc,Phi_c_des,epsilon_gdl,tau,epsilon_mc,epsilon_c,e,Re,i0_c_ref,kappa_co,kappa_c,a_slim,b_slim,a_switch,Pc_des,Sa,Phi_a_des
0,338.012705,222706.888248,2.70438,0.596459,0.682645,1.108024,0.300078,0.249759,4.0,0.000002,61.268747,24.459121,1.585349,0.053492,0.915491,0.767234,202706.888248,1.3,0.5
1,338.590419,222706.888248,2.70438,0.596459,0.682645,1.108024,0.300078,0.249759,4.0,0.000002,61.268747,24.459121,1.585349,0.053492,0.915491,0.767234,202706.888248,1.3,0.5
2,338.012705,176645.241238,2.70438,0.596459,0.682645,1.108024,0.300078,0.249759,4.0,0.000002,61.268747,24.459121,1.585349,0.053492,0.915491,0.767234,156645.241238,1.3,0.5
3,338.012705,222706.888248,2.75499,0.596459,0.682645,1.108024,0.300078,0.249759,4.0,0.000002,61.268747,24.459121,1.585349,0.053492,0.915491,0.767234,202706.888248,1.3,0.5
4,338.012705,222706.888248,2.70438,0.414280,0.682645,1.108024,0.300078,0.249759,4.0,0.000002,61.268747,24.459121,1.585349,0.053492,0.915491,0.767234,202706.888248,1.3,0.5


In [30]:
pd.DataFrame(samples).to_pickle("../data/raw/sobol_sampling_configurations_hyper_box_undertermined_restricted.pkl")

## We read the saved configurations and start the simulations

In [ ]:
samples = pd.read_pickle('../data/raw/sobol_sampling_configurations_hyper_box_undertermined_restricted.pkl')
samples.shape

We split the original set of samples (it's not always necessary)

In [ ]:
split_dejvis, split_nathaly, split_camila = np.array_split(samples, 3)
print(f'shape1 {split_dejvis.shape}, shape2 {split_nathaly.shape} and shape3 {split_camila.shape}')

In [ ]:
save_filepath = "../data/raw/sobol_sampling_configurations_hyper_box_undertermined_restricted_results.pkl"
results = get_polarisation_curve_samples(sampled_parameters=split_nathaly.to_dict(orient='records'), fixed_parameters = build_fixed_parameters(), save_path=save_filepath, save_every=10)#


✅ Saved 10 samples to ../data/raw/sobol_sampling_configurations_hyper_box_undertermined_restricted_results.pkl
✅ Saved 20 samples to ../data/raw/sobol_sampling_configurations_hyper_box_undertermined_restricted_results.pkl
✅ Saved 30 samples to ../data/raw/sobol_sampling_configurations_hyper_box_undertermined_restricted_results.pkl
